In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
posts = pd.read_csv('../data/raw_data/posts.csv')
users = pd.read_csv('../data/raw_data/users.csv')

In [9]:
users.sample(5)

,uid,first_name,last_name,sex,is_leader
4223,250902630,Катерина ...,Дмитриева,1,0
40065,234339819,Андрей ...,Кравчук,2,0
34140,60462737,Наталья ...,Гращенкова,1,0
5841,299757144,Гена ...,Корбан,2,0
2018,23677721,Юлия ...,Петрова,1,0


In [10]:
posts.sample(5)

,from_id,owner_id,text,comments,likes,reposts
1993405,223623635,223623635,СЦЕНАРНАЯ КУХНЯ.\n\nТакеши Китано успешно соче...,0,25,0
2239427,960692,960692,А вот кому нужен котик-мягкий животик? :),0,8,5
1783469,98271106,8010810,Ахуенный фристайл!!!,0,0,0
2354424,171567,1826463,С днем рождения!,0,0,0
723905,229443471,229443471,дыхание осени...,0,0,0


In [50]:
valid_users = np.unique(users['uid'])

In [52]:
posts = posts[posts['from_id'].isin(valid_users)]
posts['text'] = posts['text'].astype('str')
data = posts.groupby(['from_id'], sort=True).agg({'text': ' '.join,
                                                  'comments': 'sum',
                                                  'likes': 'sum',
                                                  'reposts': 'sum'})

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
users.index = users.uid
users.drop('uid', axis=1, inplace=True)

In [78]:
data['is_leader'] = users.loc[data.index]['is_leader']

In [80]:
X = data.drop('is_leader', axis=1)
Y = data['is_leader'].values

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer()
model = LogisticRegression()


model.fit(tfidf.fit_transform(X['text']), Y)

In [89]:
preds = model.predict(tfidf.transform(X['text']))

In [90]:
submission = pd.DataFrame({'is_leader':preds}, index=X.index)

submission.to_csv('../data/submission/train_submission.csv')